# Pipeline de préparation des données pour la détection d’oiseaux morts

Ce notebook décrit l’ensemble du workflow de préparation des données utilisé pour l’entraînement du modèle de détection d’oiseaux morts à partir d’images aériennes de haute résolution.

En raison de la grande taille des images originales et de la très petite taille relative des oiseaux, un apprentissage direct sur les images brutes est peu efficace. Un pipeline de pré-traitement en plusieurs étapes est donc mis en place, comprenant :

- l’analyse statistique du jeu de données initial,
- le découpage des images en tuiles de 512×512 pixels,
- l’augmentation de données,
- la fusion des jeux de données générés,
- l’analyse finale du dataset utilisé pour l’apprentissage.

L’objectif est d’améliorer la visibilité des objets, d’augmenter la diversité des échantillons et de renforcer la capacité de généralisation du modèle.


In [7]:
import os
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter


## 1. Analyse du jeu de données initial

In [8]:
def analyze_dataset(root):
    stats = []
    for split in ["train", "valid", "test"]:
        img_dir = os.path.join(root, split, "images")
        lbl_dir = os.path.join(root, split, "labels")

        if not os.path.exists(img_dir):
            continue

        images = glob.glob(os.path.join(img_dir, "*.jpg")) + glob.glob(os.path.join(img_dir, "*.png"))
        labels = glob.glob(os.path.join(lbl_dir, "*.txt"))

        n_boxes = 0
        n_empty = 0
        for f in labels:
            with open(f) as file:
                lines = file.readlines()
                if len(lines) == 0:
                    n_empty += 1
                n_boxes += len(lines)

        stats.append([split, len(images), len(labels), n_boxes, n_empty])

    return pd.DataFrame(stats, columns=["Split", "Images", "Labels", "Boxes", "Empty"])


In [3]:
ORIGINAL_DATASET = "C:\\Users\\DELL\\Documents\\S9\\Projet_Pic\\dead-bird-detection\\exports"

df_original = analyze_dataset(ORIGINAL_DATASET)
df_original

,Split,Images,Labels,Boxes,Empty
0,train,106,106,118,0
1,valid,32,32,34,0
2,test,14,14,16,0


## 2. Découpage des images (Tiling)


In [4]:
print("Lancement du script de découpage...")
os.system("python test_split.py")
print("Découpage terminé.")


Lancement du script de découpage...
Découpage terminé.


In [6]:
TILED_DATASET = "C:\\Users\\DELL\\Documents\\S9\\Projet_Pic\\dead-bird-detection\\PIC_2_test_tiled_512_v2"
df_tiled = analyze_dataset(TILED_DATASET)
df_tiled


,Split,Images,Labels,Boxes,Empty
0,train,0,0,0,0
1,valid,0,0,0,0
2,test,0,0,0,0


## 3. Augmentation de données

In [ ]:
print("Lancement du script d’augmentation...")
os.system("python augmentation.py")
print("Augmentation terminée.")


In [19]:
AUG_DATASET = "C:\\Users\\DELL\\Documents\\S9\\Projet_Pic\\dead-bird-detection\\augmented_dataset"

df_aug = analyze_dataset(AUG_DATASET)
df_aug


,Split,Images,Labels,Boxes,Empty
0,train,615,615,646,0
1,valid,249,249,249,0
2,test,107,107,107,0


In [5]:
AUG_DATASET = "dead-bird-detection/augmented_dataset"
df_aug = analyze_dataset(AUG_DATASET)
df_aug


,Split,Images,Labels,Boxes,Empty


## 4. Fusion des datasets

In [ ]:
COMBINED_ROOT = "dead-bird-detection/combined_dataset"

def merge_datasets(src_roots, dst_root):
    for split in ["train"]:
        for root in src_roots:
            img_dir = os.path.join(root, split, "images")
            lbl_dir = os.path.join(root, split, "labels")

            out_img = os.path.join(dst_root, split, "images")
            out_lbl = os.path.join(dst_root, split, "labels")

            os.makedirs(out_img, exist_ok=True)
            os.makedirs(out_lbl, exist_ok=True)

            for f in glob.glob(os.path.join(img_dir, "*")):
                os.system(f'copy "{f}" "{out_img}"')

            for f in glob.glob(os.path.join(lbl_dir, "*")):
                os.system(f'copy "{f}" "{out_lbl}"')

merge_datasets([TILED_DATASET, AUG_DATASET], COMBINED_ROOT)
print("Fusion terminée.")


In [ ]:
df_final = analyze_dataset(COMBINED_ROOT)
df_final


## 5. Visualisation


In [ ]:
plt.figure(figsize=(7,5))
plt.bar(df_final["Split"], df_final["Images"])
plt.title("Nombre d’images après préparation")
plt.ylabel("Images")
plt.grid(True)
plt.show()


In [ ]:
import os
from pathlib import Path
from PIL import Image

# Chemin vers le dataset
dataset_path = Path("dataset")

splits = ["train", "val", "test"]

for split in splits:
    images_path = dataset_path / split / "images"
    labels_path = dataset_path / split / "labels"
    
    image_files = list(images_path.glob("*.jpg")) + list(images_path.glob("*.png"))
    label_files = list(labels_path.glob("*.txt"))
    
    print(f"\n=== Split: {split} ===")
    print(f"Nombre d'images: {len(image_files)}")
    
    # Vérifier la dimension des images
    wrong_dim = []
    for img_file in image_files:
        with Image.open(img_file) as img:
            if img.size != (512, 512):
                wrong_dim.append(img_file.name)
    if wrong_dim:
        print(f"Images avec mauvaise dimension (pas 512x512): {len(wrong_dim)}")
        print(wrong_dim[:5], "..." if len(wrong_dim) > 5 else "")
    else:
        print("Toutes les images sont de 512x512 ✅")
    
    # Analyse des labels
    total_annotations = 0
    empty_labels = 0
    out_of_bounds = 0
    
    for lbl_file in label_files:
        with open(lbl_file, "r") as f:
            lines = f.readlines()
            if len(lines) == 0:
                empty_labels += 1
            for line in lines:
                total_annotations += 1
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"Format incorrect: {lbl_file.name}")
                    continue
                _, x, y, w, h = map(float, parts)
                if not (0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                    out_of_bounds += 1
    
    print(f"Nombre total d'annotations: {total_annotations}")
    print(f"Images sans annotations: {empty_labels}")
    print(f"Annotations hors limites [0,1]: {out_of_bounds}")
